# **Pipelines Activity**

_John Andrew Dixon_

---

**Setup**


In [2]:
# Import necessary modules

# For working with the data
import pandas as pd

# For performing a TTS

# For scaling numerical features and encoding nominal features

# For creating column selectors and column transformers


In [3]:
# Remote URL that has the data
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSdnb9XcAnl91bdZYxoJQgIapMW6SLkfr3DYGwnpBOIw-rkw-5j_3b0JLx01282OBAKVUCUJnq8OAUR/pub?output=xlsx"

# Read in the data and verify
df = pd.read_excel(url)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 16 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   name                                             77 non-null     object 
 1   Manufacturer                                     77 non-null     object 
 2   type                                             68 non-null     object 
 3   calories per serving                             70 non-null     float64
 4   grams of protein                                 77 non-null     int64  
 5   grams of fat                                     69 non-null     float64
 6   milligrams of sodium                             76 non-null     float64
 7   grams of dietary fiber                           77 non-null     float64
 8   grams of complex carbohydrates                   77 non-null     float64
 9   grams of sugars                   

---

## **Tasks**

> **Question**: _How well can the calories be predicted based on the Manufacturer, cereal type, grams of fat, grams of sugars, and weight in ounces per one serving of the cereal?_

### **Define features (X) and target (y).**
- X should only include the Manufacturer, cereal type, grams of fat, grams of sugars, and weight in ounces columns.
- y should be calories per serving

### **Train test split the data to prepare for machine learning.**

### **Identify each feature as numerical, ordinal, or nominal.**

### **Use pipelines and column transformers to complete the following tasks:**
- Impute any missing values. Use the ‘mean’ strategy for numeric columns and the ‘most_frequent’ strategy for categorical columns.
- One-hot encode the nominal features.
    - Be sure to include the arguments: sparse=False AND handle_unknown='ignore' when creating your OneHotEncoder.
- Scale the numeric columns.

### **All preprocessing steps should be contained within a single preprocessing object.**
- Include the arguments: remainder='drop' OR remainder='passthrough' when creating your ColumnTransformer

### **Use your preprocessing object to transform your data appropriately, avoiding data leakage, to make it ready for modeling. Show the resulting NumPy array output.**
- The .fit() and .transform() methods should ONLY be used with the resulting preprocessing object, NOT with any individual transformer.